In [22]:
import pandas as pd
import arrow
from dateutil import tz
import numpy as np
import plotly.express as px


In [23]:
system_power = 5 #kW
system_capacity = 13.5 #kWh
cycling_charge = 0.07 #$/kWh
solar_system_size = 5 #kW

avg_fcas_availability = 0.5
fcas_safety_factor = 0.7

rert_price = 15 #$/kWh

years = [2016, 2017, 2018, 2019, 2020]
states = ['SA', 'VIC', 'NSW', 'QLD']
results = {
    'SA': {2016: {}, 2017: {}, 2018: {}, 2019: {}, 2020: {}},
    'VIC': {2016: {}, 2017: {}, 2018: {}, 2019: {}, 2020: {}},
    'NSW': {2016: {}, 2017: {}, 2018: {}, 2019: {}, 2020: {}},
    'QLD': {2016: {}, 2017: {}, 2018: {}, 2019: {}, 2020: {}},
    }

In [24]:
# Step 1, general savings for spot exposed customer, taken from amberelectric.com.au front page. Then subtracted lost solar savings due to lower average FiT

# TODO make solar losses a bit more sophisticated

amber_dmo_adjustment = {
    'SA': (1850-1600)-(.05*0.8*solar_system_size*365),
    'VIC': (1325-1100)-(.05*0.6*solar_system_size*365),
    'NSW': (1475-1225)-(.05*0.9*solar_system_size*365),
    'QLD': (1575-1400)-(.05*1*solar_system_size*365)
    }
for k,v in amber_dmo_adjustment.items():
    for year in years:
        results[k][year]['dmo_adjustment'] = v


In [25]:
# Step 2, peak pricing

for state in states:
    df = pd.read_csv("files/{}12016-2020.csv".format(state),parse_dates=['SETTLEMENTDATE'])
    for year in years:
        results[state][year]['wholesale_peak'] = 0
    df.index = df['SETTLEMENTDATE']
    DFList = [group[1] for group in df.groupby(df.index.date)]
    for day in DFList:
        day = day[day['RRP'] > 1000]
        if len(day) > 0:
            price = day['RRP'].mean()/1000 #  average peak price converted to $/kWh 
            duration = min(2, len(day)/2)/2 # hours of peak pricing generated against
            profit = system_power * price * duration
            results[state][day.index.year[0]]['wholesale_peak'] += profit


In [26]:
# Step 3, precharging during high cloud cover days
for state in states:
    df = pd.read_csv("files/{}12016-2020.csv".format(state),parse_dates=['SETTLEMENTDATE'])
    for year in years:
        results[state][year]['precharging'] = {'count': 0, 'value': 0}
    df.index = df['SETTLEMENTDATE']
    DFList = [group[1] for group in df.groupby(df.index.date)]
    for day in DFList:
        daylight_hours = day.between_time('8:00', '16:00')
        daylight_hours_cloud = daylight_hours['CLOUD'].mean()
        if daylight_hours_cloud > 0.7:
            day = day[day['RRP'] < 1000] # don't include offseting super peak prices, this is done in wholesale_peak
            precharging_price = day['RRP'].nlargest(2).mean()/1000 - day['RRP'].nsmallest(2).mean()/1000
            if precharging_price > cycling_charge:
                results[state][day.index.year[0]]['precharging']['value'] += precharging_price * system_capacity / 2
                results[state][day.index.year[0]]['precharging']['count'] += 1


In [27]:
# Step 4, FCAS Revenue
for state in states:
    df = pd.read_csv("files/{}12016-2020.csv".format(state),parse_dates=['SETTLEMENTDATE'])
    df.index = df['SETTLEMENTDATE']
    df['FCASTOTALRRP'] = (df['RAISE6SECRRP'] + df['RAISE60SECRRP'] + df['RAISE5MINRRP'] + df['LOWER6SECRRP'] + df['LOWER60SECRRP'] + df['LOWER5MINRRP'])
    DFList = [group[1] for group in df.groupby(df.index.year)]
    for y in DFList:
        results[state][y.index.year[0]]['fcas'] = y['FCASTOTALRRP'].sum() / (2*1000) * avg_fcas_availability * fcas_safety_factor * system_power

results['SA'][2020]['fcas'] = results['SA'][2020]['fcas']/2 # adjustment for nemweb data not taking into account administered price cap which overinflates results during Feb'2020 Heywood interconnector outage


In [28]:
# Step 4, network support savings
network_support_savings = 50
for state in states:
    for year in years:
        results[state][year]['network_support'] = network_support_savings

In [29]:
# Step 5, RERT
for state in states:
    for year in years:
        results[state][year]['rert'] = 0
results['VIC'][2019]['rert'] = 6 * rert_price * system_power
results['SA'][2019]['rert'] = 2 * rert_price * system_power
results['NSW'][2020]['rert'] = 6 * rert_price * system_power
results['VIC'][2020]['rert'] = 2 * rert_price * system_power

In [30]:
results
for state in states:
    sa1 = {'years': [], 'spot_savings': [], 'peak_earnings': [], 'precharging': [], 'fcas': [], 'network_support': []}
    for year in years:
        sa1['years'].append(year)
        sa1['spot_savings'].append(results[state][year]['dmo_adjustment'])
        sa1['peak_earnings'].append(results[state][year]['wholesale_peak'])
        sa1['precharging'].append(results[state][year]['precharging']['value'])
        sa1['fcas'].append(results[state][year]['fcas'])
        sa1['network_support'].append(results[state][year]['network_support'])
    dfsa = pd.DataFrame(data=sa1)
    dfsa.to_csv('results/{}dynamic-plan-savings.csv'.format(state))
    wide_df = dfsa
    fig = px.bar(wide_df, x="years", y=["spot_savings", "peak_earnings", "precharging", "fcas", "network_support"], title="{} Savings".format(state))
    fig.show()

In [31]:
results
for state in states:
    sa1 = {'years': [], 'rert': [], 'fcas': []}
    for year in years:
        sa1['years'].append(year)
        sa1['fcas'].append(results[state][year]['fcas'])
        sa1['rert'].append(results[state][year]['rert'])
    dfsa = pd.DataFrame(data=sa1)
    dfsa.to_csv('results/{}-hive-savings.csv'.format(state))
    wide_df = dfsa
    fig = px.bar(wide_df, x="years", y=["fcas", "rert"], title="{} Savings".format(state))
    fig.show()